In [2]:
import os
import plaidml.keras
import pandas as pd
import dask.dataframe as dd
import nltk
import numpy as np
import swifter
nltk.download('stopwords')

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pedromadruga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [240]:
%%time

dataset_name = "jobindex_cropped_bigger"

import os.path
extension = os.path.basename(f'../data/interim/{dataset_name}.csv')

print(extension)

df = pd.read_csv(f'../data/interim/{dataset_name}.csv')

df_init = df[['title', 'company', 'description','location','link','ratings_link', 'source', 'date']]

df_init['merged'] = (
                        df_init['title'].fillna('') + ' '
                        + df_init['company'].fillna('') + ' '
#                         + df_init['description'].fillna('')+ ' '
                        + df_init['location'].fillna('')+ ' '
#                         + df_init['link'].astype(str).fillna('')+ ' '
#                         + df_init['ratings_link'].fillna('')+ ' '
#                         + df_init['source'].fillna('')+ ' '
#                         + df_init['description'].fillna('')+ ' '
                        + df_init['date'].astype(str).fillna('')
                        )

df_init['merged'] = df_init['merged'].astype(str)

df_init.head(100)

jobindex_cropped_bigger.csv
CPU times: user 137 ms, sys: 46.7 ms, total: 184 ms
Wall time: 320 ms


<timed exec>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,title,company,description,location,link,ratings_link,source,date,merged
0,Østervang søger en social- og sundhedshjælper ...,Ikast-Brande Kommune,"Ikast-Brande Kommune, KlovborgStillingen er på...",Klovborg,http://www.ikast-brande.dk/,/virksomhed/26512/ikast-brande-kommune#show_ra...,NaN,2020-03-10,Østervang søger en social- og sundhedshjælper ...
1,Ørnereden i Lystrup-Elsted Dagtilbud søger en ...,Aarhus Kommune,DII Ørnereden søger en pædagog til en fast...,Lystrup,https://www.jobindex.dk/jobannonce/jobnet/8794964,/virksomhed/8742/aarhus-kommune#show_rating,Jobcenter,2020-02-27,Ørnereden i Lystrup-Elsted Dagtilbud søger en ...
2,Ørepropper / Høreværn,NaN,Her finder du vores udvalg af ørepropper o...,SoundStoreXL.com A/S,https://www.soundstorexl.com/produkter/6089-oe...,NaN,SoundStoreXL.com A/S,2020-03-24,Ørepropper / Høreværn SoundStoreXL.com A/S 20...
3,Ørelægen i Virum søger lægesekretær,Olin Øre Næse Hals Klinik v/Helle Birgitte Dah...,Olin Øre Næse Hals Klinik v/Helle Birgitte Dah...,NaN,http://www.olinklinik.dk/,NaN,NaN,2020-03-03,Ørelægen i Virum søger lægesekretær Olin Øre N...
4,Øreklinikken dalgas plads søger sygeplejerske,Øreklinikken Dalgasplads,Øreklinikken Dalgas Plads søger sygeplejer...,Herning,https://www.jobindex.dk/jobannonce/jobnet/8807347,NaN,Jobcenter,2020-03-05,Øreklinikken dalgas plads søger sygeplejerske ...
...,...,...,...,...,...,...,...,...,...
95,sælger søges - Garantiløn på 22.000 kr.,TELEWORKS A/S,Her er jobbet for dig med hang til at tjen...,København Ø,https://www.jobindex.dk/jobannonce/jobnet/8794992,NaN,Jobcenter,2020-02-27,sælger søges - Garantiløn på 22.000 kr. TELEWO...
96,sygeplejerske/bioanalytiker,Region Midtjylland,Kan du spejle dig i værdier som tværfaglig...,Aarhus N,https://rm.emply.net/recruitment/vacancyAd.asp...,/virksomhed/22700/region-midtjylland#show_rating,Region Midtjylland,2020-03-13,sygeplejerske/bioanalytiker Region Midtjylland...
97,sygeplejerske ved Plastik og Brystkirurgi,Region Midtjylland,Vi udvider kapaciteten på vores mikrokirur...,Aarhus N,https://rm.emply.net/recruitment/vacancyAd.asp...,/virksomhed/22700/region-midtjylland#show_rating,Region Midtjylland,2020-03-09,sygeplejerske ved Plastik og Brystkirurgi Regi...
98,sygeplejerske til sengeafsnit,Region Midtjylland,Sengeafsnit med udsigt søger sygeplejerske...,Aarhus N,https://rm.emply.net/recruitment/vacancyAd.asp...,/virksomhed/22700/region-midtjylland#show_rating,Region Midtjylland,2020-03-10,sygeplejerske til sengeafsnit Region Midtjylla...


In [241]:
# distilbert-multilingual-nli-stsb-quora-ranking
from sentence_transformers import SentenceTransformer, util
import numpy as np

embedder = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')

In [242]:
corpus_unprocessed = df_init['merged'][:1000]

# df_init['bow'] = df_init['merged'][:1000].swifter.apply(pp_text)

corpus_unprocessed.tail(50)

950    To bibliotekarer DB søges til projektansættels...
951    To bevillingskonsulenter med interesse for udv...
952    To basissygeplejesker, der vil være med fra st...
953    To ambitiøse pædagoger søges til den integrere...
954    To Social- og sundhedsassistenter til Center B...
955    To SSA´er søges til fast dagvagt på Gundsø Oms...
956    To SSA til Solbakken Slagelse Kommune Korsør 2...
957    To Hørekonsulenter søges til Videnscenter for ...
958    To DW-konsulenter, der kan gøre data til et st...
959    Tjenerelev – Tivoli Hotel & Congress Center, K...
960           Tjenerelev Comwell Køge Strand  2020-02-28
961    Tjenere til restaurant med uovertruffen udsigt...
962    Tjenere til Kragerup Gods Kragerup Gods Ruds V...
963    Tjenere søges!! CAFE' MARINA ApS Hvide Sande 2...
964    Tjener/serveringsmedarbejder søges til restaur...
965            Tjener-bartender Lava Aarhus C 2020-03-02
966    Tjener søges til Hotel Ballumhus Hotel Ballumh...
967    Tjener søges Ritz Café &

In [243]:
## preprocessing text

import re
import numpy as np
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.snowball import DanishStemmer
da_stop_words = stopwords.words('danish')
import lemmy

lemmatizer = lemmy.load("da")
stemmer = DanishStemmer()

all_descriptions = np.array(df_init['merged'])
all_descriptions_wo_stopwords = []

# def preprocess_text(merged_features):
#     all_descriptions_wo_stopwords = []
        
#     ## remove stopwords
#     for sentence in merged_features:
#         removed_stop_words = [x for x in sentence.split() if not x in da_stop_words]
#         result = ' '.join(removed_stop_words)
#         all_descriptions_wo_stopwords.append(result)
    
#     return all_descriptions_wo_stopwords


def pp_text(text):
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    
    textblob = TextBlob(text)
#     print('\n initial length: ', len(textblob.words))
    singles = [stemmer.stem(word) for word in textblob.words]
#     print('\n singles length: ', len(singles))
    no_stop_words = [word for word in singles if not word in da_stop_words]
#     print('\n no stop words length: ', len(no_stop_words))
    joined_text = " ".join(no_stop_words)
    final_text = lemmatizer.lemmatize("", joined_text)
#     print('\n final text length: ', len(final_text), '\n\n\n')
    
    return final_text[0]

corpus = corpus_unprocessed.swifter.apply(pp_text)
df_init['title_processed'] = df_init['title'].swifter.apply(pp_text)

<ipython-input-243-aea200523e07>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_init['title_processed'] = df_init['title'].swifter.apply(pp_text)


In [244]:
import multiprocessing

corpus_embeddings = embedder.encode(corpus_unprocessed, convert_to_tensor=False)
# test_embeddings = embedder.encode(df_init['title'][:100], convert_to_tensor=False)

In [245]:
import numpy as np
from sklearn import get_config
from sklearn.metrics import pairwise_distances_chunked, pairwise_distances, pairwise_kernels, pairwise
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow import convert_to_tensor
import csv
from tqdm import tqdm

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(corpus)


x, y = corpus_embeddings.shape
index_cols = np.arange(0, x, 1).tolist()


# %time gen = pairwise_distances_chunked(X, metric='cosine', working_memory=1)
# gen2 = pairwise_distances_chunked(X,  metric='cosine')

%time gen2 = pairwise_distances_chunked(X, metric='cosine', n_jobs=-1)
# genY = pairwise_distances_chunked(corpus_embeddings, metric='cosine')
%time genY2 =  pairwise_distances_chunked(corpus_embeddings, metric='cosine', n_jobs=-1)

# %time gen3 = pairwise_distances_chunked(X, metric='cosine', n_jobs=-1)

# for i in gen:
#     print(i)

# %time cos_scores = util.pytorch_cos_sim(corpus_embeddings, corpus_embeddings)[0]

# for index, value in enumerate(gen2):
#     print('ITEM:', item.shape)
#     print('index', index)
#     print('\n')

# print(next(gen3))
print('\n\n')

with open("output_tfidf.csv", "w") as fp:
    wr = csv.writer(fp, delimiter=',', quoting = csv.QUOTE_ALL)
    wr.writerow(index_cols)
    for chunk in tqdm(gen2):
        for item in chunk:
            wr.writerow(item)
        
with open("output_bert.csv", "w") as fp:
    wr = csv.writer(fp, delimiter=',', quoting = csv.QUOTE_ALL)
    wr.writerow(index_cols)
    for chunk in tqdm(genY2):
        for item in chunk:
            wr.writerow(item)

# result = next(gen2)[random_index]
# print('shape', result.shape)
# print('\n\n')

# result2 = next(genY2)[random_index]

0it [00:00, ?it/s]

CPU times: user 62 µs, sys: 5 µs, total: 67 µs
Wall time: 33.9 µs
CPU times: user 36 µs, sys: 2 µs, total: 38 µs
Wall time: 21 µs





1it [00:00,  1.00it/s]
1it [00:01,  1.35s/it]


In [246]:
import tensorflow as tf 

def search(job_title):
    direct_search_results = [df_init['title_processed'][:1000].index[df_init['title_processed'][:1000].str.contains(word, case=False)].values for word in job_title.split()]
    return list(set(direct_search_results[0]).intersection(*direct_search_results))

title = "sygeplejerske psykiatrisk"
# print('corpus:',  corpus[1])
# print('corpus unp:',  corpus_unprocessed[1])
# print('title: ', df['title'][1])
print('\n')

search_query = title
print('searching for: ', pp_text(search_query))

search_results = search(pp_text(search_query))
## IMPORTANT: could improve with text_preprocessing
## OR could just use word embeddings

agg_results_indexes = []

print('\n DIRECT RESULTS')
for result in search_results:
#     print('result: ', result)
    print('- ', df_init['title'][result])
#     print(df_init['description'][result])
    agg_results_indexes.append(result)

    

# def convert_sparse_matrix_to_sparse_tensor(X):
#     coo = X.tocoo()
#     indices = np.mat([coo.row, coo.col]).transpose()
#     return tf.SparseTensor(indices, coo.data, coo.shape)

# convert_sparse_matrix_to_sparse_tensor(X)

# query_embedding_tfidf = vectorizer.fit_transform(search_query)
# query_embedding_tfidf = embedder.encode(search_query, convert_to_tensor=False)
# distances_tfidf = util.pytorch_cos_sim(query_embedding_tfidf, convert_sparse_matrix_to_sparse_tensor(X))[0]

# y = pairwise.cosine_similarity(X,query_embedding_tfidf)
# print(query_embedding_tfidf.shape)
# print(X.shape)


agg_results = []
agg_indexes = []

for index, result in enumerate(search_results):

    distances = pairwise_kernels(X, X[result], n_jobs=-1, metric='cosine')
    # print(distance)
    
    flat_distances = np.array(distances).flatten()
    
    for index, distance in enumerate(flat_distances):
#         agg_indexes.append(index)
        
        agg_results.append({
        'distance': float(distance),
        'index': index,
        'result_title': df_init['title'][index],
        'description': df_init['description'][index]
        })

query_embedding_tfidf = vectorizer.fit_transform(search_query.split(' '))
# distances_tfidf = pairwise_kernels(X, X[520], n_jobs=-1, metric="cosine")

# since this was run on a CPU, there was no need to convert to tensor
query_embedding_bert = embedder.encode(search_query, convert_to_tensor=True)

distances_bert = util.pytorch_cos_sim(query_embedding_bert, corpus_embeddings)[0]
# distances_tfidf = util.pytorch_cos_sim(query_embedding_tfidf, corpus_embeddings)[0]

sorted_bert = sorted(range(len(distances_bert)), key=lambda k: distances_bert[k], reverse=True)[:20]
# sorted_tfidf = sorted(range(len(distances_tfidf)), key=lambda k: distances_tfidf[k], reverse=True)[:20]

sorted_tfidf = sorted(agg_results, key=lambda k: k['distance'], reverse=True)[:20]

# print('\n\n BERT:')
# # for item in sorted_bert:
#     # remove direct results
# for index in agg_results_indexes:
#     for item in sorted_bert:
#         if (item != index):
#             print(df_init['title'][item])
#             print(item)
#             print('\n')
    

# print(query_embedding_bert)
print('\n\n TFIDF:')

# sorted_tfidf_indexes = []
for item in sorted_tfidf:
    agg_indexes.append(item['index'])

tdfidf_recommendation_indexes = [x for x in agg_indexes if x not in agg_results_indexes]
# bert_recommendation_indexes = [x for x in sorted_bert if x not in agg_results_indexes]


for index in tdfidf_recommendation_indexes:
    print('- ', df_init['title'][index])
    print(agg_results[index]['distance'])
#     print(df_init['distance'][index])
    
print('\n\n BERT:')
bert_recommendation_indexes = [x for x in sorted_bert if x not in agg_results_indexes]
for index in bert_recommendation_indexes:
    print('- ', df_init['title'][index])
    print(distances_bert[index])

# for index in agg_results_indexes:
#     for item in sorted_tfidf:
# #         print(item['index'] == index.astype(np.int32))
#         if (item['index'] != index.astype(np.int32)):
#     #     print(df_init['title'][item])
#             print(item['result_title'])
#             print(item['distance'])
#             print(item['index'])
        #     print(item['description'])
#             print('\n')
#     print(distances[item])




searching for:  sygeplejersk psykiatrisk

 DIRECT RESULTS
-  Ældrepsykiatrisk afsnit P 201 i Odense søger sygeplejersker til faste stillinger


 TFIDF:
-  Ældrepsykiatrisk afsnit P201 i Odense søger social- og sundhedsassistenter til faste stillinger
0.8187916935319689
-  Ældrepsykiatrisk afsnit P201 i Odense søger social- og sundhedsassistenter til faste stillinger
0.5918539200414299
-  Vedligeholdelsesmand søges (Odense S)
0.39534329744343283
-  Vi opnormerer fortsat. Neurokirurgisk sengeafsnit U-1 søger 2 sygeplejersker til faste stillinger pr. 01.04.2020, eller ...
0.34797936472671775
-  Tyskunderviser til vores erhvervskunde i Odense
0.327647142553221
-  Vil du give udsatte børn i Odense ferieoplevelser for livet?
0.28428641493097034
-  Vil du give udsatte børn i Odense ferieoplevelser for livet?
0.2836157619310635
-  Vil du give udsatte børn i Odense ferieoplevelser for livet?
0.2768770950166601
-  Virksomhedkonsulent, målrettet byggeri/håndværk til Virksomhed og Arbejdskrat 1


In [247]:
query_embedding_tfidf = embedder.encode(search_query, convert_to_tensor=False)

type(query_embedding_tfidf)

numpy.ndarray

In [248]:
%%time

random_index = 520

print('\n\n')
print(f"title: {df_init['title'][random_index]}")
print(f"location: {df_init['location'][random_index]}")
print(f"company: {df_init['company'][random_index]}")
print(f"description: {df_init['description'][random_index]}")


print('\n #### RESULTS: #### \n\n')

df_tfidf = pd.read_csv('output_tfidf.csv')  
df_bert = pd.read_csv('output_bert.csv')




tfidf_distances = df_tfidf[str(random_index)].values
stuff = [None] * x 

for result in search_results:
    tfidf_distances = df_tfidf[str(result)].values
#     print(tfid)
    for index,distance in enumerate(tfidf_distances):
        stuff[index]({"value": distance, "result_id": result}) 


for result in search_results:
    results_appended.append({
        "result": result,
        "values": df_tfidf[str(result)].values
    })

# print(stuff)





title: Vi søger en ergoterapeut, der brænder for at arbejde med rehabilitering og genoptræning i et tværfagligt, udviklende og omskiftligt miljø
location: nan
company: Fredensborg Kommune
description: Fredensborg KommunePilebo er en rehabiliteringsafdeling med midlertidige pladser, genoptræningspladser og pleje-/akutplads – der er i alt 24 pladser.Som ergoterapeut på Pilebo starter dagen med fælles morgenmøde, hvor både sygeplejersker, SSA’er og terapeuter er til stede. Herefter varetages forskellige opgaver. Det kan være en ADL-træning/vurdering med borgere vi har i huset. Det kan fx. være en ortopædkirurgisk borger som har behov for træning af brug af hjælpemidler eller en geriatrisk og/eller neurologisk borger, hvor der er behov for vurdering af kognitionen i aktivitet m.m. Det kan også være kognitive screeninger fx. MoCA, træning af OE eller udføre dysfagiscreening evt. i forbindelse med borgers morgenmåltid. 

 #### RESULTS: #### 




TypeError: 'NoneType' object is not callable

In [249]:
# sorted_tfidf = sorted(range(len(tfidf_distances)), key=lambda k: tfidf_distances[k], reverse=False)[:5]

df_bert = pd.read_csv('output_bert.csv')  
bert_distances = df_bert[str(random_index)].values
sorted_bert = sorted(range(len(bert_distances)), key=lambda k: bert_distances[k], reverse=False)[:5]

# for index_value in sorted_tfidf:
#     print(df_init["title"][index_value])
#     print(df_init["location"][index_value])
#     print(df_init["company"][index_value])
#     print('\n')
    
print('BERT: \n\n')

for index_value in sorted_bert:
    print(df_init["title"][index_value])
    print(df_init["location"][index_value])
    print(df_init["company"][index_value])
    print('\n')

BERT: 


Vi søger en ergoterapeut, der brænder for at arbejde med rehabilitering og genoptræning i et tværfagligt, udviklende og omskiftligt miljø
nan
Fredensborg Kommune


Ældrecenter Dalvangen afdeling C søger Social- og sundhedsassistent til fast stilling i dagvagt med vagt hver 3. weekend
Glostrup
Glostrup Kommune


Vikarer / Afløsere søges til Plejecenteret Sønderhaven
Måløv
Plejecenter Sønderhaven


Vikariat og timeafløser til Rehabiliteringscenter Grøndal inde
Haslev
Faxe Kommune


Vikariat og timeafløser til Rehabiliteringscenter Grøndal inde
Haslev
Faxe Kommune


